In [1]:
%reload_ext autoreload
%autoreload 2

import jax
import jax.numpy as jnp
import fitsio

import impt
from impt.fpfs import *
from impt.fpfs.default import *

In [3]:
ndat=1000000
print("Simulating catalog with %d galaxies" %ndat)
key = jax.random.PRNGKey(212)  # Random seed is explicit in JAX
cat = jax.random.normal(key=key, shape=(ndat, ncol))

params = FpfsParams(Const=2.0)
e1F = FpfsE1(params)
e2F = FpfsE2(params)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Simulating catalog with 1000000 galaxies


In [4]:
print("loading the noise covariance")
test_fname = os.path.join(
    impt.fpfs.__data_dir__,
    "fpfs-cut32-0000-g1-0000.fits",
    )
data=fitsio.read(test_fname)
%time noise_cov=impt.fpfs.utils.fpfsCov2lptCov(data)

loading the noise covariance
CPU times: user 2.05 ms, sys: 2 µs, total: 2.05 ms
Wall time: 2.01 ms


In [5]:
print("preparing the function for noise bias factor estimation")
%time e1_noi= impt.BiasNoise(e1F, noise_cov)
print("estimating noise bias")
%time e1_noi.evaluate(cat)

preparing the function for noise bias factor estimation
CPU times: user 399 µs, sys: 0 ns, total: 399 µs
Wall time: 410 µs
estimating noise bias
CPU times: user 1.87 s, sys: 12 ms, total: 1.88 s
Wall time: 1.87 s


Array([ 0.01420485, -0.01691402,  0.00158234, ..., -0.16089322,
        0.21418488,  0.00549208], dtype=float64)

In [6]:
print("preparing the function for shear response estimation")
%time e1_res=impt.RespG1(e1F)
print("estimating shear response")
%time out=e1_res.evaluate(cat)

preparing the function for shear response estimation
CPU times: user 358 µs, sys: 35 µs, total: 393 µs
Wall time: 405 µs
estimating shear response
CPU times: user 197 ms, sys: 7.93 ms, total: 205 ms
Wall time: 195 ms
